In [1]:
def conOptDistance(x,y):
    """
    Function that computes the distance between molecules for contact
    or optical clusters
    
    Parameters:
    -----------
    x : array
        The 1D array of size 3*ats representing the first molecule
    y : array
        The 1D array of size 3*ats representing the second molecule
        
    Returns
    -------
    r : float
        The distance between x and y computed as the minimum distance
        between any two beads in the molecules
    """
    if len(x) % 3 != 0 or len(y) % 3 != 0:
        raise RuntimeError("3D array has a number of entries not divisible \
                            by 3.")
    ats = len(x)/3
    xa = np.reshape(x,[ats,3])
    ya = np.reshape(y,[ats,3])
    #return np.min(euclidean_distances(xa,ya,squared=True))    
    return np.min(cdist(xa,ya,metric='sqeuclidean'))

def conOptDistanceC(x,y):
    """
        Function that computes the distance between molecules for contact
    or optical clusters
    
    Parameters:
    -----------
    x : array
        The 1D array of size 3*ats representing the first molecule
    y : array
        The 1D array of size 3*ats representing the second molecule
        
    Returns
    -------
    r : float
        The distance between x and y computed as the minimum distance
        between any two beads in the molecules
        
    Notes
    -----
    Uses scipy.weave to incorporate a little bit of C code to see if that
    will speed things up
    """
    if len(x) % 3 != 0 or len(y) % 3 != 0:
        raise RuntimeError("3D array has a number of entries not divisible \
                            by 3.")
   
    #xa = np.reshape(x,[ats,3])
    #ya = np.reshape(y,[ats,3])
    mind = 10000.0
    support = '#include <math.h>'
    
    code = """
    int i,j;
    return_val = 0;
    double d;
    for (i = 0; i < Nx[0]/3; i++) {
       for (j = 0; j < Nx[0]/3; j++) {
           d = (x[3*i] - y[3*i]) * (x[3*i] - y[3*i])
               + (x[3*i + 1] - y[3*i + 1]) * (x[3*i + 1] - y[3*i + 1])
               + (x[3*i + 2] - y[3*i + 2]) * (x[3*i + 2] - y[3*i + 2]);
           if (d < mind){
               mind = d;            
           }
       }     
    }
    return_val = mind;
	
			"""
    mind = weave.inline(code,['x', 'y', 'mind'],
                        support_code = support, libraries = ['m'])
    #return np.min(euclidean_distances(xa,ya,squared=True))    
    return mind

In [ ]:
def conOptDistC_inpy(x,y):
    if len(x) % 3 != 0 or len(y) % 3 != 0:
        raise RuntimeError("3D array has a number of entries not divisible \
                            by 3.")
    mind = 10000.0
    for i in range(len(x)/3):
        for j in range(len(x)/3):
            